**Data Science part:**

Create a script able to scrape the trustpilot website for the following energy suppliers, making sure that you have the comments of the first 10 pages:
https://fr.trustpilot.com/review/eni.fr
https://fr.trustpilot.com/review/totalenergies.fr
Make sure that for each comment you have the date, the body, the rate and (bonus) the answer of the supplier.

In [158]:
#!pip install selenium

In [159]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

# Libraries for the last exercise (optionnal)
import os
import pandas as pd
import time
import matplotlib.pyplot as plt
import datetime
import argparse
from bs4 import BeautifulSoup
import requests

In [160]:
#!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


'/Users/dnn/anaconda3/lib/python3.11/site-packages/chromedriver_autoinstaller/120/chromedriver'

In [205]:
# set the target URL for total energies
url = "https://fr.trustpilot.com/review/totalenergies.fr"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

# Using Selenium

In [222]:
rate_path = '//*[@id="__next"]/div/div/main/div/div[4]/section/div[13]/article/div/section/div[1]/div[1]/img'

rate_element = driver.find_element(By.XPATH, rate_path)
print(rate_element.get_attribute("alt"))
rate = rate_element.get_attribute("alt")[5:6]
rate

Noté 4 sur 5 étoiles


'4'

In [225]:
rate_path  = '//*[contains(@class, "star-rating_starRating__4rrcf star-rating_medium__iN6Ty" )]//img'


rate_element = driver.find_element(By.XPATH, rate_path)
rate = rate_element.get_attribute("alt")[5:6]

rate

'S'

In [218]:
com_list = []

com_path   = '//*[contains(@class, "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn" )]'

com = driver.find_element(By.XPATH, com_path,).text


review_list = driver.find_elements( By.XPATH,'//*[contains(@class, "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl" )]',)
reviews = list(map(lambda x: x.text, review_list))   #list with all the review of the same pages
reviews[1]

for review in review_list:
    com_path   = './/*[contains(@class, "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn" )]'
    com = review.find_element(By.XPATH, com_path).text
    com_list.append(com)
print(com_list[:2])

["Tout d'abord jai recu des appels manques des que jai envoye ma demande de souscription a l'entreprise.\n\nPuis 2 jours plutard apres avoir communique au telephone avec l'entreprise, jai recu confirmation de mon abonnement. Sincerement Obiely A Tayoro", "Contact facile, l'équipe dédiée est à l'écoute, le contrat a été fait rapidement. L'interlocutrice que j'ai eu au téléphone est très attentive et professionnelle. Rappel deux jours après pour formaliser le contrat."]


# Using Beautiful Soup

In [169]:
from bs4 import BeautifulSoup
from datetime import datetime



ratings = []
replies = []




url = 'https://fr.trustpilot.com/review/' + 'eni' + '.fr?'
response = requests.get((f"{url}page={1}"))
web_page = response.text
soup = BeautifulSoup(web_page, "html.parser")

for review in soup.find_all(class_ = 'paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl'):


  # Review dates
  date = review.find(class_ = date_path)
  date = date.getText()[22:] # Remove the unnecessary part 'Date de l'expérience:'

  # Replace French month names with English month names in the date string
  for french_month, english_month in french_to_english_month.items():
      date = date.replace(french_month, english_month)    # 

  date = datetime.strptime(date, date_format) # Parse the date string into a datetime object
  dates.append(date)

  # Review ratings             
  rating = review.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild()
    #get the first child element of the found element, 
  rating = rating["alt"][5:6] # Just get the number of stars
  ratings.append(int(rating)) # Convert rating to an integer to perform operations like mean rating

  # Review comments
  comment = review.find(class_ = com_path)
  # When there is no review text, append "" instead of skipping so that data remains in sequence with other review data
  if comment == None:
    comments.append("")
  else:
    comments.append(comment.getText())

  # Review replies
  reply = review.find(class_ = "typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX")
  # Same reasoning for the replies by the supplier
  if reply == None:
    replies.append("")
  else:
    replies.append(reply.getText())
    
    
    

# Create final dataframe from lists
df_reviews = pd.DataFrame(
   list(zip(dates, comments, replies, ratings)),
   columns =['Date', 'Comment', 'Reply', 'Rating']
)



df_reviews



,Date,Comment,Reply,Rating
0,2023-04-02,Obiely Tayoro\n1 avis\nCI\nll y a 5 jours\nCel...,,1
1,2024-01-10,MM\nMonsieur MERZOUK\n5 avis\nFR\nll y a 6 jou...,,1
2,2023-12-27,FB\nfred B.\n10 avis\nFR\nll y a 6 jours\nSous...,,1
3,2023-03-17,LI\nLisa\n2 avis\nFR\nll y a 3 jours\nMauvais ...,,1
4,2023-12-02,SF\nSabri Ferchichi\n2 avis\nFR\nIl y a un jou...,,3
5,2023-12-05,AI\nAlain IMBERT\n6 avis\nFR\n9 janv. 2024\nAc...,,1
6,2023-12-01,TL\nTommy lee\n1 avis\nFR\nll y a 4 jours\nPer...,,1
7,2023-12-14,Moulinie Christel\n1 avis\nFR\nll y a 3 jours\...,,1
8,2023-07-01,JO\nJONATHAN\n5 avis\nFR\n13 janv. 2024\nDirec...,,1
9,2023-12-13,AB\nAntoine Brigitte\n4 avis\nMQ\n8 janv. 2024...,,1


In [137]:
result = webscrapping_trustpilot('totalenergies')
result

DataFrame has been saved to Reviews_totalenergies


,Date,Title,Comment,Reply,Rating
0,2024-01-16,Celerite chez TotalEnergies,Tout d'abord jai recu des appels manques des q...,,5
1,2024-01-10,Abonnement,"Contact facile, l'équipe dédiée est à l'écoute...",,5
2,2024-01-09,Souscription en ligne facile et…,Souscription en ligne facile et relativement r...,,4
3,2024-01-19,Mauvais tarif d'abonnement,Ne prenez pas votre abonnement de gaz chez eux...,,1
4,2024-01-21,Le meilleur distributeur d'électricité…,Le meilleur distributeur d'électricité et de g...,,5
...,...,...,...,...,...
195,2023-12-01,Offre promotionnelle intéressante,,,5
196,2023-11-28,Très pro...,,,5
197,2023-11-21,Excellente interlocutrice 🙏,,,5
198,2023-11-15,Très bonne.,,,5


In [189]:
# Define the format of your date string
date_format = '%d %B %Y'  # %d is day, %B is full month name, %Y is year with century as a decimal number

# Define a dictionary to map French month names to English month names
french_to_english_month = {
    'janvier': 'January',
    'février': 'February',
    'mars': 'March',
    'avril': 'April',
    'mai': 'May',
    'juin': 'June',
    'juillet': 'July',
    'août': 'August',
    'septembre': 'September',
    'octobre': 'October',
    'novembre': 'November',
    'décembre': 'December'
}



In [190]:
# Review dates
date_path  = 'typography_body-m__xgxZ_ typography_appearance-default__AAY17'

date = review.find(class_ = date_path)
print(date.getText())
date = date.getText()[22:] # Remove the unnecessary part 'Date de l'expérience:'
date


Date de l'expérience: 08 décembre 2023


'08 décembre 2023'

In [191]:
# Replace French month names with English month names in the date string
for french_month, english_month in french_to_english_month.items():
    date = date.replace(french_month, english_month)
date

'08 December 2023'

In [192]:
date = datetime.strptime(date, date_format) # Parse the date string into a datetime object
date

datetime.datetime(2023, 12, 8, 0, 0)

In [197]:
# Review ratings             
rating = review.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild()
rating

<img alt="Noté 1 sur 5 étoiles" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-1.svg"/>

In [198]:

#get the first child element of the found element, 
rating = rating["alt"][5:6] # Just get the number of stars
int(rating) # Convert rating to an integer to perform operations like mean rating

1